The purpose of this notebook is to indentify what cells were used for testing.

In [1]:
# open test_results.pkl

import pickle
import numpy as np
from core.backend import open_minian

with open('test_results.pkl', 'rb') as f:
    results = pickle.load(f)

In [2]:
# Iterate through experiments

dict = {"cross_session_same_day": [["./data/PL010/PL010_D1S1", "./data/PL010/PL010_D1S4"], ["./data/AA058/AA058_D1S1", "./data/AA058/AA058_D1S4"], ["./data/AA036/AA036_D2S1", "./data/AA036/AA036_D2S4"], ["./data/AA034/AA034_D1S1", "./data/AA034/AA034_D1S4"]],
        "cross_day_same_session": [["./data/PL010/PL010_D1S1", "./data/PL010/PL010_D8S1"], ["./data/AA058/AA058_D1S1", "./data/AA058/AA058_D5S1"], ["./data/AA036/AA036_D2S1", "./data/AA036/AA036_D6S1"], ["./data/AA034/AA034_D1S1", "./data/AA034/AA034_D7S1"]],
        "cross_day_cross_session": [["./data/PL010/PL010_D1S1", "./data/PL010/PL010_D8S4"], ["./data/AA058/AA058_D1S1", "./data/AA058/AA058_D5S4"], ["./data/AA036/AA036_D2S1", "./data/AA036/AA036_D6S4"], ["./data/AA034/AA034_D1S1", "./data/AA034/AA034_D7S4"]],
        "cross_animal": [["./data/PL010/PL010_D1S1", "./data/AA058/AA058_D1S1"], ["./data/AA058/AA058_D1S1", "./data/AA036/AA036_D2S1"], ["./data/AA036/AA036_D2S1", "./data/AA034/AA034_D1S1"], ["./data/AA034/AA034_D1S1", "./data/PL010/PL010_D1S1"]],
        "within_session": [["./data/PL010/PL010_D1S1"], ["./data/AA058/AA058_D1S1"], ["./data/AA036/AA036_D2S1"], ["./data/AA034/AA034_D1S1"]],
        "all": [["./data/PL010/PL010_D1S1", "./data/PL010/PL010_D1S4", "./data/PL010/PL010_D8S1", "./data/PL010/PL010_D8S4",
                 "./data/AA058/AA058_D1S1", "./data/AA058/AA058_D1S4", "./data/AA058/AA058_D5S1", "./data/AA058/AA058_D5S4", 
                 "./data/AA036/AA036_D2S1", "./data/AA036/AA036_D2S4", "./data/AA036/AA036_D6S1", "./data/AA036/AA036_D6S4",
                 "./data/AA034/AA034_D1S1", "./data/AA034/AA034_D1S4", "./data/AA034/AA034_D7S1", "./data/AA034/AA034_D7S4"]]}

experiments = results.keys()

def find_test_set(experiment, training_set):
        arr = dict[experiment]
        if len(arr[0]) == 1:
                for train in arr:
                        if train[0][-len(training_set):] == training_set:
                                return train[0]
        elif experiment == "all":
                pass                
        else:
                for train, test in arr:
                        if train[-len(training_set):] == training_set:
                                return test
        print(f"Error: Test set not found for {experiment} and {training_set}")

def get_ground_truth(arr):
        # Get rows 0, 2, 4 etc. from the array
        return arr[::2]
        

In [3]:
results["all"]

{'GRU': [array([0, 0, 0, ..., 0, 0, 0], dtype=int8),
  array([0, 0, 0, ..., 0, 0, 0], dtype=int8),
  array([0, 0, 0, ..., 0, 0, 0], dtype=int8),
  array([0, 0, 0, ..., 0, 0, 0], dtype=int8),
  array([0, 0, 0, ..., 0, 0, 0], dtype=int8),
  array([0, 0, 0, ..., 0, 0, 0], dtype=int8),
  array([0, 0, 0, ..., 0, 0, 0], dtype=int8),
  array([0, 0, 0, ..., 0, 0, 0], dtype=int8),
  array([0, 0, 0, ..., 0, 0, 0], dtype=int8),
  array([0, 0, 0, ..., 0, 1, 1], dtype=int8)]}

In [4]:
test_cells_dict = {}

for experiment in experiments:
    test_cells_dict[experiment] = {}
    training_sets = results[experiment].keys()
    for training_set in training_sets:
        test_cells_dict[experiment][training_set] = {}
        if experiment == "within_session" and training_set == "AA034_D1S1":
            print(training_set + " " + experiment)
        test_set = find_test_set(experiment, training_set)
        E = open_minian(test_set)["E"].load()
        test_set = test_set.split("/")[-1]
        test_cells_dict[experiment][training_set][test_set] = []
        all_unit_ids = E.unit_id.values
        verified = E.verified.values.astype(int)
        unit_ids = all_unit_ids[verified==1]
        E = E.sel(unit_id=unit_ids)
        ground_truth = get_ground_truth(results[experiment][training_set])
        # We need to match the ground truth with the results in E and return the name of the cell
        for row in ground_truth:
            # This took me a while to figure out but I made a small mistake when trying to generate the data, I used 26999 frames instead of 27000
            # Split the row into 5 equal parts
            mini_rows = np.array_split(row, 5)
            mini_indices = []
            for mini_r in mini_rows:
                found = False
                for unit_id in unit_ids:
                    if (E.sel(unit_id=unit_id).values[:-1] == mini_r).all():
                        mini_indices.append(unit_id)
                        found = True
                        break
            
                if not found:
                    raise Exception(f"Error: Cell not found in test set {test_set} for experiment {experiment} and training set {training_set}")
                
            test_cells_dict[experiment][training_set][test_set].append(mini_indices) 
    
    

Error: Test set not found for all and GRU


TypeError: stat: path should be string, bytes, os.PathLike or integer, not NoneType

In [5]:
test_cells_dict



{'cross_animal': {'AA034_D1S1': {'PL010_D1S1': [[10, 81, 52, 20, 24],
    [69, 22, 52, 80, 48],
    [50, 51, 24, 44, 52],
    [22, 63, 79, 20, 46],
    [46, 16, 88, 24, 80],
    [90, 46, 88, 50, 80],
    [74, 44, 63, 20, 10],
    [74, 51, 10, 69, 80],
    [88, 24, 16, 10, 51],
    [44, 56, 52, 48, 81],
    [79, 1, 80, 56, 81],
    [44, 1, 88, 20, 48],
    [50, 20, 24, 81, 90],
    [69, 80, 45, 91, 81],
    [63, 80, 45, 44, 24],
    [88, 1, 56, 8, 74],
    [84, 91, 50, 81, 8],
    [1, 52, 16, 50, 45],
    [24, 22, 84, 69, 79],
    [16, 84, 10, 52, 88],
    [44, 91, 79, 1, 81],
    [8, 52, 74, 80, 48],
    [79, 84, 51, 91, 10],
    [45, 44, 80, 8, 48],
    [48, 20, 56, 79, 52],
    [48, 16, 24, 69, 46],
    [81, 48, 80, 63, 51],
    [16, 45, 90, 79, 69],
    [22, 56, 8, 24, 50],
    [46, 84, 74, 56, 91]]},
  'AA036_D2S1': {'AA034_D1S1': [[59, 51, 143, 21, 132],
    [51, 88, 80, 67, 21],
    [127, 143, 117, 103, 90],
    [103, 59, 91, 46, 90],
    [84, 58, 16, 143, 91],
    [88, 51, 105, 

In [6]:
results.keys()

dict_keys(['cross_animal', 'cross_day_cross_session', 'cross_day_same_session', 'cross_session_same_day', 'within_session'])

In [7]:
# Pickle the dictionary
with open('test_cells_dict.pkl', 'wb') as f:
    pickle.dump(test_cells_dict, f)